In [17]:
import pandas as pd
import io
import re
import csv
import json
import numpy as np
from numpy import savetxt, loadtxt, asarray
from tqdm import tqdm
import math
import ast
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import seaborn as sns
import keras
from keras import backend as K
from progressbar import ProgressBar
import matplotlib.pyplot as plt
from keras import Sequential, Model
from random import randrange
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Input
import time
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.externals import joblib



from plot_figures import save_model

pbar = ProgressBar()
tqdm.pandas()

In [2]:


max_blocks = 37 #(36 + 1 FC layer at the end)
nb_param =7
nb_hw_param = 12

inversed = True
only_inversed = False

if inversed and not only_inversed:
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/with_inversed_hw"
elif only_inversed:
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/only_inversed_hw"
else : 
    path_entire_model= "/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/results_best_models/multiply/no_inversed_hw"

BATCH_SIZE = 2048
SHUFFLE_BUFFER_SIZE = 100

DATASET_SIZE = 168308
train_size = int(0.90 * DATASET_SIZE) #135 805
test_size = int(0.10 * DATASET_SIZE) # 15 089

nb_training_batches = train_size //BATCH_SIZE +1
nb_test_batches = test_size //BATCH_SIZE +1

In [3]:

def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.asarray(ast.literal_eval(array_string))

# Load Proccessed Neural Networks (without HyperParameters)

In [6]:
path_raw_val = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/good_data/parsed_nondups_val_3.csv'
path_processed_val_nn = '/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/data/good_data/val7_from1600.csv'




In [7]:
#%pycache
tin= time.clock()   
val = pd.read_csv(path_processed_val_nn,converters={'NN_dataframe': from_np_array})
tfin= time.clock()
print(f'Total time : {(tfin-tin)/60} min')

Total time : 0.1831873 min


# Complete to zeros (max shape)

In [8]:
def get_shape(arr):
    return arr.shape[0]

def add_zero_blocks(arr):
    zero_blocks = np.zeros((max_blocks-arr.shape[0],arr.shape[1]))
    return np.append(arr, zero_blocks, axis=0)

val["nb_blocks"] = val["NN_dataframe"].apply(lambda x : get_shape(x))
val['NN_dataframe']= val['NN_dataframe'].apply(lambda x : add_zero_blocks(x))




# Get y_train, y_val


In [9]:
# To retrieve y_train / y_test

#train_hw = pd.read_csv(path_raw_train)
val_hw = pd.read_csv(path_raw_val)

#y_train = np.array(train_hw["total_power"].tolist())
#y_train=y_train.reshape(y_train.shape[0],-1)

y_val = np.array(val_hw["total_power"].tolist())
y_val=y_val.reshape(y_val.shape[0],-1)

--------
# HW PARAM ALONE

In [10]:
#X_train_hw_param  = train_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
#X_train_hw_param['tot_mac'] = X_train_hw_param['mac_num']*X_train_hw_param['mac_array_num']

X_val_hw_param  = val_hw[['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size']]
X_val_hw_param['tot_mac'] = X_val_hw_param['mac_num']*X_val_hw_param['mac_array_num']

#X_train_hw_param.head()

# Shapes of the blocks

In [11]:
#X_train_shape = np.array(train["nb_blocks"])
X_val_shape = np.array(val["nb_blocks"])

#X_val_shape

# Inv

In [12]:
def inv(x):
    return 1/(1+x)


if inversed : 
#nb_hw_param = 6
    nb_hw_param = 12

    X_val_hw_param['1/mac_num'] = X_val_hw_param['mac_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/mac_array_num'] = X_val_hw_param['mac_array_num'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/data_bits'] = X_val_hw_param['data_bits'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/sram_size'] = X_val_hw_param['sram_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/max_filter_size'] = X_val_hw_param['max_filter_size'].progress_apply(lambda x : inv(x))
    X_val_hw_param['1/tot_mac'] = X_val_hw_param['tot_mac'].progress_apply(lambda x : inv(x))



100%|██████████| 24274/24274 [00:00<00:00, 402739.46it/s]


In [13]:
X_val_hw_param.head()

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac,1/mac_num,1/mac_array_num,1/data_bits,1/sram_size,1/max_filter_size,1/tot_mac
0,37.0,2.0,128.0,199936.0,4098.0,74.0,0.026316,0.333333,0.007752,0.000005,0.000244,0.013333
1,91.0,2.0,256.0,119936.0,2048.0,182.0,0.010870,0.333333,0.003891,0.000008,0.000488,0.005464
2,74.0,2.0,256.0,139904.0,1024.0,148.0,0.013333,0.333333,0.003891,0.000007,0.000976,0.006711
3,70.0,2.0,512.0,80000.0,512.0,140.0,0.014085,0.333333,0.001949,0.000012,0.001949,0.007092
4,103.0,2.0,256.0,199936.0,4098.0,206.0,0.009615,0.333333,0.003891,0.000005,0.000244,0.004831


## HW Normalize

In [19]:

scaler = joblib.load('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/hw_scaler.pkl') 
X_val_hw_param_norm = scaler.transform(X_val_hw_param )



In [20]:
pd.DataFrame(X_val_hw_param_norm).head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.595863,-0.489793,-0.841296,0.754353,2.061669,-0.412974,-0.169928,0.706748,1.441544,-1.095801,-1.155747,0.067051
1,0.371807,-0.489793,-0.647015,-0.147927,0.332994,-0.297455,-0.595003,0.706748,0.103602,-0.409372,-0.778893,-0.427118
2,0.067170,-0.489793,-0.647015,0.077282,-0.530500,-0.333822,-0.527201,0.706748,0.103602,-0.654222,-0.026105,-0.348810
3,-0.004509,-0.489793,-0.258452,-0.598345,-0.962247,-0.342379,-0.506529,0.706748,-0.569281,0.447012,1.477272,-0.324896
4,0.586844,-0.489793,-0.647015,0.754353,2.061669,-0.271784,-0.629517,0.706748,0.103602,-1.095801,-1.155747,-0.466906


------------

# NN TREATEMENT ALONE

In [22]:


#X_train_nn =np.array(train['NN_dataframe'].tolist())
X_val_nn = np.array(val['NN_dataframe'].tolist())

## NN Normalize

### Divde columns by standard deviation

In [24]:


means = []
std = np.loadtxt('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/std.csv', delimiter=',')

#X_train_nn_norm=X_train_nn.copy()
X_val_nn_norm = X_val_nn.copy()
for i in range(nb_param):
    #means.append(np.mean(X_train_nn[:,:,i]))
    #X_train_nn[:,:,i]=-means[i]
    #std.append(np.std(X_train_nn[:,:,i]))
    if std[i]!=0:
        #X_train_nn_norm[:,:,i]/= std[i]
        X_val_nn_norm[:,:,i]/= std[i]  

#savetxt('std.csv', std, delimiter=',')
#data = loadtxt('std.csv', delimiter=',')

In [25]:
std

array([2.82669167e+08, 4.74753544e+05, 1.35528508e+05, 1.37473533e+05,
       3.10471238e+02, 1.21999774e+01, 3.94031395e-01])

# Add blocks with zeros

In [26]:


X_val_blocks = []
for sha in X_val_shape:
    ones = np.ones((sha, 1))
    zeros = np.zeros((37-sha,1))
    X_val_blocks.append(np.concatenate((ones,zeros)))
X_val_blocks = np.array(X_val_blocks)# Full Dataset (HW_params + NN arrays)

# Full Dataset (HW_params + NN arrays)

In [27]:
#NORMED
#train_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_train_nn_norm, X_train_hw_param_norm), y_train))
test_dataset_norm = tf.data.Dataset.from_tensor_slices(((X_val_nn_norm, X_val_hw_param_norm), y_val))

#train_dataset_norm = tf.data.Dataset.from_tensor_slices(( (X_train_blocks,X_train_nn_norm, X_train_hw_param_norm), y_train) )
#test_dataset_norm = tf.data.Dataset.from_tensor_slices(( (X_val_blocks,X_val_nn_norm, X_val_hw_param_norm), y_val) )

#train_dataset_norm = train_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE)
test_dataset_norm = test_dataset_norm.shuffle(SHUFFLE_BUFFER_SIZE)

#train_dataset_norm = train_dataset_norm.batch(BATCH_SIZE)
test_dataset_norm = test_dataset_norm.batch(BATCH_SIZE)


# Get loss/model

## Models

In [28]:
last_layer = 128

input_blocks = Input(shape=(max_blocks, 1),dtype='float32', name='blocks')
input_nn = Input(shape=(max_blocks, nb_param), dtype='float32', name='input_nn')



output_nn = layers.LSTM(128, return_sequences=True)(input_nn)
output_nn=layers.Dense(last_layer,  activation='relu')(output_nn)
#output_nn = tf.keras.layers.multiply([output_nn, input_blocks])
output_nn =tf.keras.layers.Lambda( lambda x: K.sum(x, axis=1))(output_nn)


model_hw = tf.keras.Sequential([
     layers.Dense(128, activation='relu', input_shape=(nb_hw_param,)),
     layers.Dense(last_layer , activation='linear')
 ])


concat = tf.keras.layers.multiply([output_nn, model_hw.output])
concat = tf.keras.layers.Concatenate()([concat,output_nn, model_hw.output])

output = tf.keras.layers.Dense(units=32, activation='relu')(concat)
output = tf.keras.layers.Dense(units=32, activation='relu')(output)
output = tf.keras.layers.Dense(units=1, activation='relu')(output)
full_model = tf.keras.Model(inputs=[input_nn, model_hw.input], outputs=[output])
#full_model = tf.keras.Model(inputs=[input_blocks, input_nn, model_hw.input], outputs=[output])



print(f'PARAMS : {full_model.count_params()}')

PARAMS : 117729


In [30]:
checkpoint_path = '/Users/roxanefischer/Desktop/LSTM_3mult_32_117729_param_0.141_error/LSTM_3mult_32_117729_param'
full_model.load_weights(checkpoint_path)
#results = full_model.evaluate(test_dataset_norm)

In [31]:
def loss(target_power, predicted_power):
  return tf.math.abs((target_power - predicted_power)/target_power)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=nb_training_batches,
    decay_rate=0.95)


optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [32]:
#%pycache


full_model.compile(optimizer=optimizer, loss=loss)

full_model.evaluate((test_dataset_norm))

t5= time.clock()


#print(f'Eval Full Model : {(t5-t4)/60} min')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
12/12 [==============================] - 4s 358ms/step - loss: 0.1409


In [34]:
full_model.evaluate((X_val_nn_norm, X_val_hw_param_norm), y_val)

759/759 [==============================] - 13s 18ms/step - loss: 0.1409


0.14092466235160828

In [37]:
hw_centers = loadtxt('/Users/roxanefischer/Desktop/single_path_nas/single-path-nas/HAS/params/hw_centroids.csv', delimiter=',')
hw_centers_unscale = scaler.inverse_transform(hw_centers)

In [38]:
# data_boths : puissance de 2
columns = ['mac_num', 'mac_array_num', 'data_bits', 'sram_size', 'max_filter_size', 'tot_mac', '1/mac_num', '1/mac_array_num', '1/data_bits', '1/sram_size', '1/max_filter_size', '1/tot_mac']
pd.DataFrame(hw_centers_unscale, columns=columns)

,mac_num,mac_array_num,data_bits,sram_size,max_filter_size,tot_mac,1/mac_num,1/mac_array_num,1/data_bits,1/sram_size,1/max_filter_size,1/tot_mac
0,75.964785,2.486868,833.547106,93668.176140,649.951653,185.341827,0.019902,0.305932,0.001698,0.000012,0.001693,0.008893
1,117.008541,3.264456,1456.113216,84107.293832,2240.018941,379.168287,0.012062,0.275778,0.000913,0.000013,0.000554,0.004806
2,60.880135,2.490219,195.103228,96333.481311,1411.857139,156.285856,0.023805,0.308794,0.006110,0.000012,0.000764,0.010976
3,86.742132,3.225397,982.058957,274859.966984,1883.697506,262.708980,0.020185,0.267869,0.001942,0.000004,0.000740,0.007445
4,20.773761,18.059963,1050.057114,133104.908250,2013.275122,397.087634,0.077908,0.066835,0.001899,0.000010,0.000708,0.006738
5,7.690948,2.447323,545.758813,115240.581936,1580.626232,18.294372,0.120950,0.304974,0.004008,0.000011,0.001033,0.057184
6,57.662028,2.640210,372.444101,96072.130567,3540.588412,153.255594,0.026937,0.300399,0.004419,0.000012,0.000295,0.011918
7,160.470151,26.022932,921.348064,181151.430322,2037.905301,4079.854238,0.006711,0.039559,0.002542,0.000008,0.000669,0.000269
8,91.468883,17.594773,916.100438,274721.784205,2106.914422,1397.397085,0.016733,0.064086,0.002380,0.000004,0.000657,0.001044
9,51.294797,2.228711,158.466006,108974.387647,512.059000,113.409224,0.027574,0.318951,0.006840,0.000011,0.001949,0.013119


In [42]:
ii=0
for ii in range(len(hw_centers)):
    print()
    print(f'index : {ii}')
    print(f'hw_center :{hw_centers[ii]}')
   # print(f'hw_center_unscale :{hw_centers_unscale[ii]}')
    hw_param_array = np.array([hw_centers[ii],]*X_val_hw_param_norm.shape[0])
    full_model.evaluate((X_val_nn_norm, hw_param_array), y_val)



index : 0
hw_center :[ 0.10237858 -0.42623102  0.2295995  -0.44418862 -0.84591852 -0.2938803
 -0.34642427  0.45720505 -0.65637991  0.38035292  1.08133546 -0.21178732]
759/759 [==============================] - 14s 18ms/step - loss: 0.2937
